In [1]:
!pip install PyPDF2
!pip install pytesseract pdf2image pillow
!sudo apt-get install tesseract-ocr
!sudo apt-get install tesseract-ocr-rus
!sudo apt-get install -y poppler-utils
!pip install ultralytics
!pip install markdownify
!pip install pix2tex -qq
!pip install openai
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.1 MB/s eta 0:00:00ta 0:00:01
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-rus
0 upgraded, 1 newly installed, 0 to remove and 30 not upgraded.
Need to get 1274 kB of archives.
After this operation, 3877 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-rus all 1:4.00~git30-7274cfa-1 [1274 kB]
Fetched 1274 kB in 0s (11.6 MB/s)          
Selecting previously unselected package tesseract-ocr-rus.
(Reading database ... 116071 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-rus_1%3a4.00~git30-7274cfa-1_all

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
open_router_api_key = user_secrets.get_secret("OPENROUTER_API_KEY")
aws_key_id = user_secrets.get_secret("AWS_ACCESS_KEY_ID")
aws_secret_key = user_secrets.get_secret("AWS_SECRET_ACCESS_KEY")

In [3]:
import io
import boto3
from PIL import Image

# Create a boto3 client for S3
s3 = boto3.client('s3',
#                   region_name="ru-central1-a", 
                  endpoint_url="https://storage.yandexcloud.net",
                  aws_access_key_id=aws_key_id,
                  aws_secret_access_key=aws_secret_key)

# Upload to S3 bucket
bucket_name = 'afana-propdoc-production'
def save_img_to_s3(s3, bucket_name, s3_key, cropped_img_pil):
    in_mem_file = io.BytesIO()
    cropped_img_pil.save(in_mem_file, format='JPEG') 
    # Important: move the file pointer to the beginning
    in_mem_file.seek(0)
    s3.put_object(Bucket=bucket_name, Key=s3_key, Body=in_mem_file, ContentType='image/jpeg')  # Adjust ContentType if needed

def upload_zip_file_to_s3(s3, bucket_name, s3_key, zip_file_path):
    """
    Uploads an existing ZIP file to an S3 bucket.

    :param bucket_name: Name of the S3 bucket
    :param s3_key: S3 key (path) where the ZIP file will be uploaded
    :param zip_file_path: Local path to the ZIP file to be uploaded
    """
    try:
        # Open the ZIP file in binary mode
        with open(zip_file_path, 'rb') as zip_file:
            # Upload the ZIP file to S3
            s3.upload_fileobj(zip_file, bucket_name, s3_key)
            print(f"ZIP file uploaded successfully to s3://{bucket_name}/{s3_key}")

    except Exception as e:
        print(f"Failed to upload ZIP file to S3: {e}")

# response = s3.list_objects_v2(Bucket=bucket_name)
# if 'Contents' in response:
#     for obj in response['Contents']:
#         print(f"File: {obj['Key']}")
# else:
#     print("No files found in the bucket.")
    
    
# # Read the image back from the S3 bucket
# response = s3.get_object(Bucket=bucket_name, Key=s3_key)
# image_data = io.BytesIO(response['Body'].read())
# image_pil12 = Image.open(image_data)

# # Optionally display the image (on systems where display is available)
# from IPython.display import display

# # Display the image in the notebook
# display(image_pil12)

In [4]:
import os
import base64
from openai import OpenAI

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    
prompt = """Представь, что ты эксперт в строительстве и соответственно в строительных нормативных документах. Ты знаком с такими документами как: "ГОСТ 5781-82, ГОСТ 21.101-2020, ГОСТ 27751-2014, Постановление_Правительства_РФ_от_16_09_2020_N_1479_Об_утверждении, СанПиН 2.1.2.2645-10, СанПиН 2.2.1, СНиП 12-01-2004 (СП 48.13330.2019), СП 20.13330.2016, СП 50.13330.2012, СП 52.13330.2016, СП 22.13330.2016_«СНиП_2_02_01_83_Основания_зданий_и_сооружений», СП 54.13330.2016_«СНиП_31_01_2003_Здания_жилые_многоквартирные», СП 50.13330.2012 Тепловая защита зданий, Постановление Правительства РФ-27.05.2022, СП-51.13330.2011, СП-60.13330.2020, СП.13130.2023". Тщательно переобдумай их на протяжении 1-2 месяцев с перерывами на хороший сон в 8 часов. Что изображено на следующем рисунке:"""

def call_openai_with_image(image_path):
    # Initialize OpenAI client with OpenRouter base URL
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=open_router_api_key
    )

    # Encode the image
    base64_image = encode_image(image_path)

    try:
        # Make the API call
        response = client.chat.completions.create(
            model="openai/gpt-4-vision-preview",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ]
        )

        # Return the generated content
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

In [5]:
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
def calculate_iou(box1, box2):
    poly1 = Polygon([(box1[0], box1[1]), (box1[2], box1[1]), (box1[2], box1[3]), (box1[0], box1[3])])
    poly2 = Polygon([(box2[0], box2[1]), (box2[2], box2[1]), (box2[2], box2[3]), (box2[0], box2[3])])
    
    if not poly1.intersects(poly2):
        return 0.0

    intersection_area = poly1.intersection(poly2).area
    union_area = poly1.union(poly2).area

    return intersection_area / union_area

def merge_overlapping_boxes(boxes, classes, scores, iou_threshold=0.9):
    merged_boxes = []
    merged_classes = []
    merged_scores = []
    
    for i, box1 in enumerate(boxes):
        keep = True
        for j, box2 in enumerate(merged_boxes):
            iou = calculate_iou(box1, box2)
            if iou > iou_threshold:
                if scores[i] > merged_scores[j]:
                    merged_boxes[j] = box1
                    merged_classes[j] = classes[i]
                    merged_scores[j] = scores[i]  
                keep = False
                break
        if keep:
            merged_boxes.append(box1)
            merged_classes.append(classes[i])
            merged_scores.append(scores[i]) 
    
    return merged_boxes, merged_classes, merged_scores


# boxes = result.boxes.xyxyn.tolist() 
# classes = result.boxes.cls.tolist() 
# scores = result.boxes.conf.tolist() 

# merged_boxes, merged_classes, merged_scores = merge_overlapping_boxes(boxes, classes, scores, iou_threshold=0.95)

In [ ]:
import pytesseract # test
from PIL import Image
import os
import cv2
import numpy as np
from tqdm.notebook import tqdm
from pdf2image import convert_from_path
import PyPDF2
import shutil
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, Colors  # Assuming you're using these custom libraries for annotations
import shutil
from pix2tex import cli as pix2tex
# from IPython.display import HTML, Math

latex_ocr_model = pix2tex.LatexOCR()
s3 = boto3.client('s3',
#                   region_name="ru-central1-a", 
                  endpoint_url="https://storage.yandexcloud.net",
                  aws_access_key_id=aws_key_id,
                  aws_secret_access_key=aws_secret_key)

# Upload to S3 bucket
bucket_name = 'afana-propdoc-production'

# Load your YOLO model
yolo_model = YOLO("/kaggle/input/yolo-10b-doclaynet/yolov10b-doclaynet.pt")

# OCR function to extract text from cropped images
def image_to_text(image):
    """Extract text from an image using Tesseract OCR."""
    return pytesseract.image_to_string(image, lang='rus') 

# Function to format sections into markdown
def format_markdown_section(class_name, text, img_path=''):
    """Format the extracted text into markdown based on its section type."""
    if class_name == "Title":
        return f"# {text.strip()}\n\n"
    elif class_name == "Section-header":
        return f"## {text.strip()}\n\n"
    elif class_name == "Page-header":
        return f"> **Page Header**: {text.strip()}\n\n"
    elif class_name == "Page-footer":
        return f"> **Page Footer**: {text.strip()}\n\n"
    elif class_name == "List-item":
        return f"- {text.strip()}\n"
    elif class_name == "Table":
        # Markdown doesn't support tables well, so just include plain text
        return f"**Table**:\n![{text.strip()}]({img_path})\n\n"
    elif class_name == "Formula":
        return f"$$ {text.strip()} $$\n\n"
    elif class_name == "Picture":
        # Placeholder for pictures
        return f"![{text.strip()}]({img_path})\n\n"
    else:  # For "Text" and other types
        return f"{text.strip()}\n\n"

pdf_directory = "/kaggle/input/product-hack-docs/"
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith(".pdf")]


for file_name in tqdm(pdf_files):
    md_output_dir = f"/kaggle/working/{file_name}/md_from_pdf/"
    unstructured_output_dir = f"/kaggle/working/{file_name}/unstructured_data/"
    
    pdf_document = os.path.join(pdf_directory, file_name)
    
    os.makedirs(md_output_dir, exist_ok=True)
    os.makedirs(unstructured_output_dir, exist_ok=True)

    
    
        

    with open(pdf_document, "rb") as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(reader.pages)
        
        for page_num in range(num_pages):
            markdown_output_file = os.path.join(md_output_dir, f"page_{page_num + 1}.md") 

            with open(markdown_output_file, 'w', encoding='utf-8') as md_file:

                page = reader.pages[page_num]


                images = convert_from_path(pdf_document, first_page=page_num + 1, last_page=page_num + 1)
                image = images[0]
                result = yolo_model.predict(image, verbose=False)[0]

                height, width = result.orig_shape[0], result.orig_shape[1]


                label_count = {
                    "Formula": 0,
                    "Picture": 0,
                    "Table": 0,
                }

                boxes = result.boxes.xyxyn.tolist()  # bounding boxes
                classes = result.boxes.cls.tolist()  # class predictions
                scores = result.boxes.conf.tolist()  # confidence scores

                merged_boxes, merged_classes, merged_scores = merge_overlapping_boxes(boxes, classes, scores, iou_threshold=0.95)

                labels_boxes = list(zip(merged_classes, merged_boxes))

                sorted_labels_boxes = sorted(labels_boxes, key=lambda x: x[1][1])  # Sort by y_min (the second value in box)

                for label, box in sorted_labels_boxes:
                    label = int(label)  # Convert label to integer
                    class_name = result.names[label]  # Get the class name for the label
                    x_min, y_min, x_max, y_max = int(box[0] * width), int(box[1] * height), int(box[2] * width), int(box[3] * height)
                    img = np.array(image)
                    cropped_img = img[y_min:y_max, x_min:x_max]

                    if class_name in label_count:
                        # Increment the label count for unique filenames
                        label_count[class_name] += 1
                        # Save the cropped image for record-keeping (optional)
                        output_filename = f"image_{label_count[class_name]}.png"
                        output_pages_path = os.path.join(unstructured_output_dir, f'page_{page_num + 1}/')
                        os.makedirs(output_pages_path, exist_ok=True)
                        output_sections_path = os.path.join(output_pages_path, f'{class_name}/')
                        os.makedirs(output_sections_path, exist_ok=True)
                        output_path = os.path.join(output_sections_path, output_filename)
                        cv2.imwrite(output_path, cropped_img)
                    cropped_img_pil = Image.fromarray(cropped_img)

                    if class_name == 'Formula':
                        extracted_text = latex_ocr_model(cropped_img_pil)
                    elif class_name == 'Picture' or class_name == 'Table':
                        extracted_text = call_openai_with_image(output_path)
                    else:
                        extracted_text = image_to_text(cropped_img_pil)
                    if class_name != 'Picture' and class_name != 'Table' and class_name != 'Formula':
                        markdown_section = format_markdown_section(class_name, extracted_text)
                    else:
                        s3_key = f"s3://afana-propdoc-production/docs_extracted_data/{file_name}/page_{page_num+1}/{class_name}/image_{label_count[class_name]}.png"
                        markdown_section = format_markdown_section(class_name, extracted_text, img_path=s3_key)
                        save_img_to_s3(s3=s3, bucket_name=bucket_name, s3_key=s3_key, cropped_img_pil=cropped_img_pil)


                    md_file.write(markdown_section)

    shutil.make_archive(f'{file_name}', 'zip', f'/kaggle/working/{file_name}/')
    upload_zip_file_to_s3(s3=s3, bucket_name=bucket_name, s3_key=f'processed_archived_documents/{file_name}.zip', zip_file_path=f'{file_name}.zip')

In [ ]:
shutil.make_archive(f'processed_data', 'zip', f'/kaggle/working/')